In [ ]:
from keras.src.saving import load_model

model = load_model("data/ckpt_best_high_preprocessing/best.keras")

In [ ]:
from src.augmentations import augment_image
from src.data_loader import load_dataframe, create_datasets

df = load_dataframe("data/cleaned_metadata.csv")

train_dataset, test_dataset = create_datasets(
    df,
    test_size=0.1,
    batch_size=64,
    augment_fn=augment_image
)

In [ ]:
from src.data_loader import denormalize_labels
from src.utils.plotting import create_predictions_visualization
import matplotlib.pyplot as plt

train_dataset_iter = iter(train_dataset)
X_test, y_test = next(train_dataset_iter)
X_test, y_test = X_test.numpy(), y_test.numpy()
y_test = denormalize_labels(y_test)

y_pred = denormalize_labels(model.predict(X_test))

points = 10
create_predictions_visualization(y_test[:points], y_pred[:points], "assets/gametable.png", arrow_length=10)
plt.title("Predictions on Training Data")
plt.show()

In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd

errors = y_test - y_pred
error_df = pd.DataFrame(errors, columns=["x_error", "y_error", "theta_error"])
print(f"Mean Squared Error: {np.mean(np.square(errors))}")
print(f"Root Mean Squared Error: {np.sqrt(np.mean(np.square(errors)))}")
print(f"Mean Absolute Error: {np.mean(np.abs(errors))}")
print(f"Max Error: {np.max(np.abs(errors))}")
print(f"Min Error: {np.min(np.abs(errors))}")
print(f"Standard Deviation: {np.std(errors)}")
sns.histplot(error_df, bins=50, kde=True)
plt.xlabel("Error Value")
plt.ylabel("Frequency")
plt.title("Error Distribution")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image as mpimg
import os
import seaborn as sns


def create_error_kde(y_test, y_pred, background_img_path, cmap='viridis', levels=50, alpha=0.6):
    if not os.path.exists(background_img_path):
        print(f"Background image not found: {background_img_path}")
        return

    background_img = mpimg.imread(background_img_path)
    image_height, image_width, _ = background_img.shape

    errors = np.linalg.norm(y_pred - y_test, axis=1)
    x_vals = y_test[:, 0]
    y_vals = y_test[:, 1]

    plt.figure(figsize=(10, 8))
    plt.imshow(background_img, extent=[0, image_width, image_height, 0], aspect='auto')

    kde_plot = sns.kdeplot(
        x=x_vals,
        y=y_vals,
        weights=errors,
        cmap=cmap,
        fill=True,
        levels=levels,
        alpha=alpha,
        thresh=0.01
    )

    cbar = plt.colorbar(kde_plot.collections[0], label="Weighted Error Density")

    plt.xlim(0, image_width)
    plt.ylim(310, 70)
    plt.xlabel("X Position")
    plt.ylabel("Y Position")
    plt.title("KDE of Prediction Error Distribution")
    plt.show()


create_error_kde(y_test, y_pred, "assets/gametable.png", cmap='magma', levels=25, alpha=0.6)

In [ ]:
model.summary()

print(f"Layer 0 type: {type(model.layers[0])}")